# Process metadata

Extracts particular fields from the full books metadata file for use in app and Datasette.

In [2]:
import pandas as pd
from sqlite_utils import Database
import json
import ast

In [3]:
df = pd.read_csv("metadata.csv")

/tmp/ipykernel_245730/195187386.py:1: DtypeWarning: Columns (50,51,52,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("metadata.csv")


In [4]:
df

,Unnamed: 0,access_restricted,aka,campaigns,contributor,date,dates,description,digitized,extract_timestamp,...,item.source_collection,item.part_of,item.repository,item.rights_advisory,item.access_advisory,composite_location,number_carrier_type,item.display_offsite,item.reproduction_number,item.resource_links
0,0,False,"['http://lccn.loc.gov/ltf90003732', 'http://ww...",[],"['blackburn, r. barry']",1895,['1895-01-01T00:00:00Z'],['Also available in digital form.'],True,2022-04-03T00:55:21.966Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,False,"['http://lccn.loc.gov/ltf90025186', 'http://ww...",[],"['anderson, l. h. (lewis h.)']",1893,['1893-01-01T00:00:00Z'],['Also available in digital form.'],True,2022-03-17T17:36:17.976Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,False,"['http://lccn.loc.gov/ltf90017756', 'http://ww...",[],"['dodge, joseph smith']",1894,['1894-01-01T00:00:00Z'],['Also available in digital form.'],True,2022-08-08T21:44:57.492Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,False,"['http://lccn.loc.gov/ltf90007547', 'http://ww...",[],"['bettelheim, bernard jean']",1840,['1840-01-01T00:00:00Z'],['Also available in digital form on the Librar...,True,2022-04-11T14:52:51.619Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,False,"['http://lccn.loc.gov/ltf90010477', 'http://ww...",[],"['vÃ¡mbÃ©ry, Ã¡rmin']",1875,['1875-01-01T00:00:00Z'],['Also available in digital form.'],True,2022-04-24T01:32:33.020Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90409,90409,False,['https://hdl.loc.gov/loc.gdc/gdclccn.19014684...,[],"['fowler, alfred']",1919,['1919-01-01T00:00:00Z'],NaN,True,2022-01-04T19:29:39.979Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90410,90410,False,"['http://lccn.loc.gov/62055408', 'http://www.l...",[],"['gates, william', 'american art association']",1924,['1924-01-01T00:00:00Z'],['Also available in digital form.'],True,2022-04-10T01:57:54.901Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90411,90411,False,"['http://lccn.loc.gov/2002418786', 'http://www...",[],"['clothey, rebecca a.', 'worden, robert l.', '...",1996,['1996-01-01T00:00:00Z'],"['""November 1996."" Includes bibliographical re...",True,2022-04-16T21:59:05.805Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90412,90412,False,"['http://lccn.loc.gov/2002418926', 'http://www...",[],"['garian, robert', 'library of congress. feder...",1997,['1997-01-01T00:00:00Z'],"['""A report prepared under an Interagency Agre...",True,2022-07-16T22:33:30.113Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.columns

Index(['Unnamed: 0', 'access_restricted', 'aka', 'campaigns', 'contributor',
       'date', 'dates', 'description', 'digitized', 'extract_timestamp',
       'group', 'hassegments', 'id', 'image_url', 'index', 'language',
       'mime_type', 'number', 'number_lccn', 'number_source_modified',
       'online_format', 'original_format', 'other_title', 'partof',
       'resources', 'segments', 'shelf_id', 'site', 'timestamp', 'title',
       'url', 'item.call_number', 'item.contributors',
       'item.created_published', 'item.date', 'item.format', 'item.language',
       'item.medium', 'item.notes', 'item.title', 'number_oclc', 'type',
       'item.digital_id', 'subject', 'item.subjects', 'location',
       'item.location', 'item.creator', 'item.contents', 'item.other_title',
       'number_preceding_items', 'partof_title', 'publication_frequency',
       'item.genre', 'location_city', 'location_country',
       'number_succeeding_items', 'number_former_id', 'number_issn',
       'item.con

In [6]:
df.loc[df["description"].notnull()]["description"].to_list()[:10]

["['Also available in digital form.']",
 "['Also available in digital form.']",
 "['Also available in digital form.']",
 "['Also available in digital form on the Library of Congress Web site.']",
 "['Also available in digital form.']",
 "['Also available in digital form.']",
 "['Also available in digital form.']",
 "['Romanized. Also available in digital form.']",
 "['Also available in digital form.']",
 "['Also available in digital form.']"]

In [7]:
df["id"]

0        http://www.loc.gov/item/ltf90003732/
1        http://www.loc.gov/item/ltf90025186/
2        http://www.loc.gov/item/ltf90017756/
3        http://www.loc.gov/item/ltf90007547/
4        http://www.loc.gov/item/ltf90010477/
                         ...                 
90409       http://www.loc.gov/item/19014684/
90410       http://www.loc.gov/item/62055408/
90411     http://www.loc.gov/item/2002418786/
90412     http://www.loc.gov/item/2002418926/
90413     http://www.loc.gov/item/2015297433/
Name: id, Length: 90414, dtype: object

In [8]:
df.loc[df["id"].str.contains("00000649")]["item.title"]

57124    Cyclopedia of classified dates, with an exhaus...
Name: item.title, dtype: object

In [10]:
df_meta = df.copy()[["id", "item.title", "item.contributors", "item.created_published", "item.date", "image_url"]]

In [11]:
df_meta["item.contributors"] = df_meta["item.contributors"].dropna().apply(ast.literal_eval).str.join('; ')
df_meta["item.created_published"] = df_meta["item.created_published"].dropna().apply(ast.literal_eval).str.join('; ')
df_meta["image_url"] = df_meta["image_url"].dropna().apply(lambda x: ast.literal_eval(x)[0])

In [12]:
df_meta.loc[df_meta["image_url"].notnull()]["image_url"]

0        https://tile.loc.gov/image-services/iiif/publi...
1        https://tile.loc.gov/image-services/iiif/publi...
2        https://tile.loc.gov/image-services/iiif/publi...
3        https://tile.loc.gov/image-services/iiif/servi...
4        https://tile.loc.gov/image-services/iiif/publi...
                               ...                        
90409    https://tile.loc.gov/image-services/iiif/servi...
90410    https://tile.loc.gov/image-services/iiif/publi...
90411    https://tile.loc.gov/image-services/iiif/publi...
90412    https://tile.loc.gov/image-services/iiif/publi...
90413    https://tile.loc.gov/image-services/iiif/publi...
Name: image_url, Length: 90414, dtype: object

In [13]:
df_meta.columns = ["url", "title", "contributors", "details", "date", "image_url"]

In [14]:
df_meta.insert(0, "id", df_meta["url"].str.extract(r"\/([a-z0-9]+)\/$"))

In [15]:
df_meta

,id,url,title,contributors,details,date,image_url
0,ltf90003732,http://www.loc.gov/item/ltf90003732/,"Art, society, and accomplishments at home and ...","Blackburn, R. Barry. [from old catalog]","Chicago, W. B. Conkey Co., 1895.",1895,https://tile.loc.gov/image-services/iiif/publi...
1,ltf90025186,http://www.loc.gov/item/ltf90025186/,"How to win; or, Sure secrets of success. Menta...","Anderson, L. H. (Lewis H.)","Chicago, National Hygienic Institute, 1893.",1893,https://tile.loc.gov/image-services/iiif/publi...
2,ltf90017756,http://www.loc.gov/item/ltf90017756/,The purpose of God.,"Dodge, Joseph Smith. [from old catalog]","Boston : Universalist Pub. House, 1894.",1894,https://tile.loc.gov/image-services/iiif/publi...
3,ltf90007547,http://www.loc.gov/item/ltf90007547/,La ruine du Talmud; eÌpitre adresseÌe au gra...,"Bettelheim, Bernard Jean, 1811-1869.","Smyrne, Impr. d'A. Damiano, 1840.",1840,https://tile.loc.gov/image-services/iiif/servi...
4,ltf90010477,http://www.loc.gov/item/ltf90010477/,Der Islam in neunzehnten Jahrhundert; eine cul...,"VÃ¡mbÃ©ry, Ãrmin, 1832-1913.","Leipzig, F.A. Brockhaus, 1875.",1875,https://tile.loc.gov/image-services/iiif/publi...
...,...,...,...,...,...,...,...
90409,19014684,http://www.loc.gov/item/19014684/,"A directory of bookplate artists, with notes c...","Fowler, Alfred, ed.","Kansas City, A. Fowler, 1919.",1919,https://tile.loc.gov/image-services/iiif/servi...
90410,62055408,http://www.loc.gov/item/62055408/,[Catalogue of the William Gates collection:,"Gates, William, 1863-1940.; American Art Assoc...",New York? 1924],1924,https://tile.loc.gov/image-services/iiif/publi...
90411,2002418786,http://www.loc.gov/item/2002418786/,China in the twenty-first century : implicatio...,"Clothey, Rebecca A.; Worden, Robert L.; Librar...","Washington, D.C. : Federal Research Division, ...",1996,https://tile.loc.gov/image-services/iiif/publi...
90412,2002418926,http://www.loc.gov/item/2002418926/,North Korea in the twenty-first century : impl...,"Garian, Robert.; Savada, Andrea Matles, 1950-;...","Washington, D.C. : Federal Research Division, ...",1997,https://tile.loc.gov/image-services/iiif/publi...


In [16]:
df_meta.to_csv("cleaned_meta.csv", index=False)